In [24]:
import sys
import os

# 将项目根目录添加到 sys.path
project_path = "../"
if project_path not in sys.path:
    # print(project_path)
    sys.path.append(project_path)

In [25]:
import os
from pathlib import Path
from datetime import datetime
from typing import Dict, List
import json

from zigent.llm.agent_llms import LLM
from zigent.actions import BaseAction, ThinkAct, FinishAct 
from zigent.agents import BaseAgent,ABCAgent
from zigent.commons import TaskPackage, AgentAct
from zigent.actions.InnerActions import INNER_ACT_KEY

from zigent.logging import DefaultLogger


In [26]:
class QuizGenerationAction(BaseAction):
    """Generate quiz questions from markdown content"""
    def __init__(self, llm: LLM) -> None:
        action_name = "GenerateQuiz"
        action_desc = "Generate quiz questions from markdown content"
        params_doc = {
            "content": "(Type: string): The markdown content to generate questions from",
            "question_types": "(Type: list): List of question types and nums to generate",
            "audience": "(Type: string): Target audience for the quiz",
            "purpose": "(Type: string): Purpose of the quiz"
        }
        super().__init__(action_name, action_desc, params_doc)
        self.llm = llm
        
    def __call__(self, **kwargs):
        content = kwargs.get("content", "")
        question_types = kwargs.get("question_types", [])
        audience = kwargs.get("audience", "")
        purpose = kwargs.get("purpose", "")
        question_types_str= ""
        for index , item  in enumerate(question_types):
            if index  > 0:
                question_types_str +="，" 
            question_types_str += item["type"] +str(item["num"])+"个"

        
        prompt = f"""
        你是一个辅助设计考卷的机器人,全程使用中文。
        你的任务是帮助用户快速创建、设计考卷，考卷以markdown格式给出,选项内容不能重复。
        
        要求：
        1. 受众群体：{audience}
        2. 考察目的：{purpose}
        3. 需要包含以下题型和数量：{question_types_str}
        4. 考卷格式要求：
        """
        # print(prompt)
        
        prompt += """
        # 问卷标题
        ---
        1. 这是判断题的题干?
            - (x) True
            - ( ) False
        # (x)为正确答案

        2. 这是单选题的题干
            - (x) 这是正确选项
            - ( ) 这是错误选项
        # (x)为正确答案

        3. 这是多选题的题干?
            - [x] 这是正确选项1
            - [x] 这是正确选项2
            - [ ] 这是错误选项1
            - [ ] 这是错误选项2
        # [x]为正确答案

        4. 这是填空题的题干?
            - R:= 填空题答案
        #填空题正确答案格式
        """
        
        prompt += f"\n请根据以下内容生成考卷：\n{content}"

       
        
        quiz_content = self.llm.run(prompt)
        return {
            "quiz_content": quiz_content,
            "audience": audience,
            "purpose": purpose,
            "question_types": question_types
        }

In [27]:
class SaveQuizAction(BaseAction):
    """Save quiz to file and return URL"""
    def __init__(self) -> None:
        action_name = "SaveQuiz"
        action_desc = "Save quiz content to file and return URL"
        params_doc = {
            "quiz_content": "(Type: string): The quiz content to save",
            "quiz_title": "(Type: string): Title of the quiz"
        }
        super().__init__(action_name, action_desc, params_doc)
        
    def __call__(self, **kwargs):
        quiz_content = kwargs.get("quiz_content", "")
        quiz_title = kwargs.get("quiz_title", "quiz")
        
        output_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        os.makedirs(output_dir, exist_ok=True)
        
        output_file = os.path.join(output_dir, f"{quiz_title}.md")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(quiz_content)
            
        return {
            "file_path": output_file,
            "quiz_url": f"/{output_file}"
        }

In [28]:
# 定义 CheckQuiz 类，继承自 BaseAgent 类
class CheckQuiz(BaseAgent):
    def __init__(
        self,
        quizType,
        quizNum,
        llm: LLM,
        actions: List[BaseAction] = [], 
        manager: ABCAgent = None,
        **kwargs
    ):
        name = quizType
        # 角色
        role = f"""你是试题检查人员，负责对{quizType}的数量和格式进行检查和纠正，确保选择题格式正确，{quizType}的数量为{str(quizNum)}"""
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager
        )


In [29]:
class QuizGeneratorAgent(BaseAgent):
    """Quiz generation agent that manages quiz creation process"""
    def __init__(
        self,
        llm: LLM,
        markdown_dir: str,
    ):
        name = "QuizGeneratorAgent"
        role = """你是一个专业的考卷生成助手。你可以根据提供的Markdown内容生成结构良好、
        内容全面的考卷。你擅长根据受众群体和考察目的设计合适的题目和题目数量。"""
        
        super().__init__(
            name=name,
            role=role,
            llm=llm,

        )
        
        self.markdown_dir = markdown_dir
        self.quiz_action = QuizGenerationAction(llm)
        self.save_action = SaveQuizAction()
        self.llm = llm
        
        self._add_quiz_example()
        
    def _add_quiz_example(self):
        """Add an illustration example for the quiz generator"""
        quiz_params = {
            "audience": "零基础",  # 水平
            "purpose": "测试Python基础知识掌握情况", # 目的
            "question_types": [
                {
                    "type":"单选题",
                    "num":1
                },        
                {
                    "type":"多选题",
                    "num":2
                },
                {
                    "type":"判断题",
                    "num":1
                },
            ]   # 题型
        }
        exp_task = json.dumps(quiz_params)
        exp_task_pack = TaskPackage(instruction=exp_task) 

        # 第一步加载文档
        act_1 = AgentAct(
            name=ThinkAct.action_name,
            params={INNER_ACT_KEY: """首先，我会加载Markdown内容，然后根据受众群体和考察目的生成考卷。"""}
        )
        obs_1 = "OK. 开始加载Markdown内容。"

        # 第二步生成试卷
        act_2 = AgentAct(
            name=self.quiz_action.action_name,
            params={
                "content": "Python基础内容...",
                "question_types":quiz_params["question_types"],
                "audience": "大学生",
                "purpose": "测试Python基础知识掌握情况"
            }
        )

        obs_2 = """# Python基础测试
        ## 单选题
        1. Python是什么类型的语言?
            - (x) 解释型
            - ( ) 编译型        
        
        ## 多选题
        1. 关于 Python 中的列表和元组，以下说法正确的是？
            - [x] 列表是可变的，元组是不可变的
            - [x] 列表和元组都可以存储任意类型的元素
            - [x] 列表和元组都支持索引和切片操作
            - [ ] 列表和元组的性能完全相同，没有区别

        2. 关于 Python 中的函数，以下说法正确的是？
            - [x] 函数可以没有返回值
            - [x] 函数可以返回多个值，实际上是通过元组实现的
            - [ ] 函数的参数必须指定类型
            - [x] 函数内部可以定义嵌套函数

        ## 判断题
        1. Python 中的函数必须至少有一个参数
            - ( ) True
            - (x) False

        # [x]、(x)为正确答案

        """

        # 第三步开始检查
        question_Aciton = []
        for item in quiz_params["question_types"]:
            sk_exp_task = f"""检查一下，生成的这个试卷是否符合要求。
            生成的试卷如下：{obs_2}
            """
            sk_exp_task_pack = TaskPackage(instruction=sk_exp_task)
            quizType= item["type"]
            quizNum = item["num"]
            _question_Aciton = CheckQuiz(quizType=quizType,quizNum = item["num"],llm = self.llm)
            sk_act_1 =  AgentAct(
                    name=ThinkAct.action_name,
                    params={INNER_ACT_KEY: f"""要求的{quizType}的数量是{quizNum}个，格式是markdown！按要求检查和修正！"""
                    },
                )
            # 第一个动作的观察结果
            sk_obs_1 = "好的，我的检查和修正完毕!"

            sk_act_2 = AgentAct(
                    name=FinishAct.action_name,
                    params={INNER_ACT_KEY: f"""经过我的检查，{quizType}的数量是{quizNum}个，格式符合要求，各类型题目之间没有重复的，题目下的选项也没有重复的！"""
                    },
                )
            
            sk_obs_2 =f"{quizType}的检查任务结束！"

            # 将动作和观察组合成序列
            sk_exp_act_obs = [(sk_act_1, sk_obs_1), (sk_act_2, sk_obs_2)]
            _question_Aciton.add_example(task = sk_exp_task_pack,action_chain=sk_exp_act_obs)
            question_Aciton.append(_question_Aciton)


        act_3 = AgentAct(
            name=self.save_action.action_name,
            params={
                "quiz_content": obs_2,
                "quiz_title": "Python基础测试"
            }
        )
        obs_3 = {"file_path": "2025-01-15_03-37-40/Python基础测试.md", "quiz_url": "/2025-01-15_03-37-40/Python基础测试.md"}

        act_4 = AgentAct(
            name=FinishAct.action_name,
            params={INNER_ACT_KEY: "考卷生成并保存成功。"}
        )
        obs_4 = "考卷生成任务完成。"


        exp_act_obs = [(act_1, obs_1), (act_2, obs_2)]

        for item in question_Aciton:
            sk_act =  AgentAct(
                        name=item.name,
                        params={"AGENT_CALL_ARG_KEY": "检查格式和题目数量，选项不能重复，将重复的选项合并起来！",
                            },
                        )
            sk_obs ="完成了检查，修正了格式和更正了重复选项！"
            exp_act_obs.append((sk_act,sk_obs))

        exp_act_obs.append((act_3, obs_3))
        exp_act_obs.append((act_4, obs_4))
  



        self.prompt_gen.add_example(
            task=exp_task_pack,
            action_chain=exp_act_obs
        )
        
    def _load_markdown_content(self) -> str:
        """Load all markdown files from directory"""
        content = []
        for root, _, files in os.walk(self.markdown_dir):
            for file in files:
                if file.endswith(".md"):
                    with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                        content.append(f.read())
        return "\n".join(content)
        
    def __call__(self, task: TaskPackage):
        """Process the quiz generation task"""
        # Parse task parameters
        params = json.loads(task.instruction)
        audience = params.get("audience", "")
        purpose = params.get("purpose", "")
        question_types = params.get("question_types", [])
        
        # Load markdown content
        content = self._load_markdown_content()
        
        # Generate quiz
        quiz_result = self.quiz_action(
            content=content,
            question_types=question_types,
            audience=audience,
            purpose=purpose,
        )
        
        # Save quiz
        save_result = self.save_action(
            quiz_content=quiz_result["quiz_content"],
            quiz_title="generated_quiz"
        )
        
        task.answer = {
            "quiz_content": quiz_result["quiz_content"],
            "quiz_url": save_result["quiz_url"]
        }
        task.completion = "completed"
        
        return task

In [30]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('ZISHU_API_KEY')
base_url = "http://192.168.12.10:8000/v1"
chat_model = "Qwen2.5-32B-Instruct-AWQ"

llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)

# 创建出题智能体
markdown_dir = "./docs"  # 指定包含Markdown文件的目录

agent = QuizGeneratorAgent(llm=llm, markdown_dir=markdown_dir)

# 定义考卷参数
quiz_params = {
    "audience": "零基础", # 受众群体
    "purpose": "测试基础知识掌握情况", # 考察目的
    "question_types": [
        {
        "type":"单选题",
         "num":10
        },        
        {
        "type":"多选题",
         "num":10
        },
        {
        "type":"判断题",
         "num":10
        },
    ] # 需要包含的题型
}

# 生成考卷
task = TaskPackage(instruction=json.dumps(quiz_params))
result = agent(task)

print("生成的考卷内容：")
print(result.answer["quiz_content"])
print(f"考卷路径: {result.answer['quiz_url']}")

生成的考卷内容：
# 中华人民共和国审计法基础知识测试

---

## 单选题

1. 审计机关对哪一类单位的财务收支情况进行审计监督？
    - ( ) A. 国有企业
    - (x) B. 国有金融机构
    - ( ) C. 股份制企业
    - ( ) D. 私营企业

2. 审计机关对哪一类单位的预算执行情况和决算进行审计监督？
    - (x) A. 本级各部门
    - ( ) B. 其他非政府单位
    - ( ) C. 私营企业
    - ( ) D. 非营利组织

3. 审计署对什么单位的财务收支进行审计监督？
    - ( ) A. 地方政府
    - (x) B. 中央银行
    - ( ) C. 私营企业
    - ( ) D. 非营利组织

4. 审计机关对哪一类单位的内部审计工作进行业务指导和监督？
    - ( ) A. 私营企业
    - ( ) B. 非营利组织
    - (x) C. 国有资源、国有资产
    - ( ) D. 地方政府

5. 审计机关对哪一类单位的财务收支进行审计监督？
    - ( ) A. 私营企业
    - ( ) B. 非营利组织
    - (x) C. 国家事业组织
    - ( ) D. 地方政府

6. 审计机关对哪一类单位的财务收支情况进行审计监督？
    - ( ) A. 私营企业
    - (x) B. 国有企业
    - ( ) C. 非营利组织
   要求：不得重复

7. 审计机关对哪一类单位的财务收支情况进行审计监督？
    - ( ) A. 私营企业
    - (x) B. 政府投资建设项目
    - ( ) C. 非营利组织
    - ( ) D. 地方政府

8. 审计机关对哪一类单位的财务收支情况进行审计监督？
    - ( ) A. 私营企业
    - (x) B. 国有资源、国有资产
    - ( ) C. 非营利组织
    - ( ) D. 地方政府

9. 审计机关对哪一类单位的财务收支情况进行审计监督？
    - ( ) A. 私营企业
    - (x) B. 国际组织援助项目
    - ( ) C. 非营利组织
    - ( ) D. 地方政府

10. 审计机关对哪一类单位的财务收支情况

In [31]:
# #出题Action

# class QuizGenerationTypeAction(BaseAction):
#     """Generate quiz questions from markdown content by type and num"""
#     def __init__(self,  llm :LLM) ->None:
#         action_name = "GenerateQuizBytypeAndNum"
#         action_desc = "Generate quiz questions from markdown content by type and num"
#         params_doc={
#             "content": "(Type: string): The markdown content to generate questions from",
#             "question_type": "(Type: string):  question type to generate",
#             "question_num":"(Type: int):  question num to generate",
#             "audience": "(Type: string): Target audience for the quiz",
#             "purpose": "(Type: string): Purpose of the quiz"
#         }
#         super().__init__(action_name, action_desc, params_doc)
#         self.llm = llm

#     def __call__(self, **kwargs):
#         content = kwargs.get("content", "")
#         question_type = kwargs.get("question_type", "")
#         question_num = kwargs.get("question_num", "")
#         audience = kwargs.get("audience", "")
#         purpose = kwargs.get("purpose", "")
#         prompt = f"""
#         你是一个辅助设计考卷的机器人,全程使用中文。
#         你的任务是帮助用户快速创建、设计考卷，考卷以markdown格式给出,选项内容不能重复。

#         要求：
#         1. 受众群体：{audience}
#         2. 考察目的：{purpose}
#         3. 题型：{question_type}
#         4. 题目数量：{question_num}
#         5. 考卷格式要求：
#         """

#         prompt += """
#         # 判断题
#         ---
#         1. 这是判断题的题干?
#             - (x) True
#             - ( ) False
#         # (x)为正确答案

#         # 单选题

#         2. 这是单选题的题干
#             - (x) 这是正确选项
#             - ( ) 这是错误选项
#         # (x)为正确答案

#         # 多选题

#         3. 这是多选题的题干?
#             - [x] 这是正确选项1
#             - [x] 这是正确选项2
#             - [ ] 这是错误选项1
#             - [ ] 这是错误选项2
#         # [x]为正确答案

#         # 填空题

#         4. 这是填空题的题干?
#             - R:= 填空题答案
#         #填空题正确答案格式
#         """
        
        